In [1]:
import sys
sys.path.append("/home/sara/lib")

In [2]:
import pandas as pd
import os
import numpy as np
import libopf_py
import pandas as pd
import time
import random

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle

from sklearn.svm import SVC
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold # import KFold

from sklearn.model_selection import cross_validate

from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [3]:
df = pd.read_csv("csv_dataset/dataframe_7emocoes com numeros.csv", header=None, sep=';')
df = df.drop([0], axis=0)
df = df.drop(columns=[0])
df = shuffle(df)
print(df)

        1      2      3      4      5      6      7      8      9      10   \
1691   82.0  212.0   83.0  264.0   87.0  316.0   93.0  367.0  102.0  417.0   
1573   86.0  216.0   90.0  266.0   98.0  314.0  103.0  361.0  112.0  408.0   
880    87.0  203.0   88.0  253.0   94.0  303.0   99.0  352.0  108.0  400.0   
1405   71.0  223.0   75.0  274.0   84.0  324.0   91.0  373.0  105.0  421.0   
1420   84.0  234.0   89.0  285.0   99.0  334.0  109.0  383.0  124.0  430.0   
1123  101.0  215.0  103.0  265.0  109.0  314.0  114.0  362.0  122.0  410.0   
273    97.0  228.0  101.0  276.0  108.0  324.0  115.0  372.0  127.0  419.0   
41     99.0  213.0  102.0  264.0  110.0  314.0  117.0  363.0  128.0  412.0   
194    97.0  214.0  101.0  264.0  108.0  313.0  114.0  362.0  125.0  409.0   
1038   82.0  232.0   88.0  280.0   96.0  324.0  103.0  370.0  114.0  415.0   
643    68.0  235.0   73.0  284.0   83.0  329.0   93.0  374.0  107.0  417.0   
120    85.0  216.0   90.0  266.0   98.0  314.0  104.0  363.0  11

In [4]:
features = df.iloc[0:1925,0:136]
print(type(features))
features = features.to_numpy()
features = features.astype('float64')

print(features)
print(len(features))

<class 'pandas.core.frame.DataFrame'>
[[ 82. 212.  83. ... 431. 230. 429.]
 [ 86. 216.  90. ... 409. 242. 406.]
 [ 87. 203.  88. ... 415. 239. 412.]
 ...
 [ 97. 216. 102. ... 427. 259. 425.]
 [ 95. 217.  98. ... 405. 235. 403.]
 [ 94. 208.  93. ... 409. 222. 405.]]
1924


In [5]:
labels = df.iloc[0:,136]
labels = labels.to_numpy()
labels = labels.astype('float64')

print(labels)
print(len(labels))

[21. 21. 12. ...  0.  9. 13.]
1924


In [6]:
n_samples = len(features)
print(n_samples)

1924


In [7]:
train_feat,test_feat,train_labels,test_labels = train_test_split(features,labels, test_size=0.20,random_state=42 )

print ("Training set:", train_feat.shape[0], "samples")
print ("Test set:", test_feat.shape[0], "samples")

Training set: 1539 samples
Test set: 385 samples


In [8]:
# adaptações ao OPF.
def opf(train_labels1, train_feat1, test_labels1, test_feat1):
        
    # OPF only supports 32 bits labels at the moment
    label_train_32 = train_labels1.astype(np.int32)
    label_test_32 = test_labels1.astype(np.int32)

    O = libopf_py.OPF()

    O.fit(train_feat1, label_train_32)

    predicted = O.predict(test_feat1)
    name = "OPF"
    acc = accuracy_score(label_test_32, predicted)
    
    return acc  

In [9]:
C=1
models = (SVC(kernel='linear', C=C, gamma='auto'),
          SVC(kernel='rbf', gamma=0.5, C=C),
          SVC(kernel='poly', degree=3, C=C, gamma='auto'),
          KNeighborsClassifier(3),
          DecisionTreeClassifier(max_depth=8),
          RandomForestClassifier(max_depth=8, n_estimators=10, max_features=1),          
          AdaBoostClassifier(),
          GaussianNB(),
         )

names = ["SVC with linear kernel","SVC with RBF kernel","SVC with polynomial (degree 3) kernel",
         "K Nearest Neighbors","Decision Tree", "Random Forest", "AdaBoost","Naive Bayes", "OPF"]


In [10]:
def kfold(X, y):
    #X_train = []
    #X_test = []
    #y_train = []
    #y_test = []
    
    kf = KFold(n_splits = 8) # Define the split - number of folds 
    kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validatorprint(kf) KFold(n_splits=2, random_state=None, shuffle=False)

    matrix_acc = pd.DataFrame()
    fold = []
    medias = []
    i = 0
    
    for train_index, test_index in kf.split(X):
    
        train_feat, test_feat = X[train_index], X[test_index]
        train_labels, test_labels = y[train_index], y[test_index]
    
        for name, clf in zip(names, models):
            clf.fit(train_feat,train_labels) #train each model
            fold.append(clf.score(test_feat,test_labels)) #evaluate each model in the test set
        res_opf = opf(train_labels, train_feat, test_labels, test_feat)
        fold.append(res_opf)
        
        #print(fold)
        data = pd.DataFrame([fold])
        matrix_acc = matrix_acc.append(data)
        fold = []
    
    return matrix_acc
    
        #X_test.append(X[test_index])   # Uso isso se quiser debugar e saber os folds.
        #X_train.append(X[train_index])
        #y_test.append(y[test_index])
        #y_train.append(y[train_index])
        
    #return X_train, X_test, y_train, y_test

In [11]:
def names_emotions(labels_num):
    i = 0
    labels = []
    while (i < 1924):
        if labels_num[i] == 0.0:
            labels.append("neutro") 
        if labels_num[i] == 2.0:
            labels.append("alegria")
        if labels_num[i] == 9.0:
            labels.append("surpresa")
        if labels_num[i] == 12.0:
            labels.append("tristeza")
        if labels_num[i] == 13.0:
            labels.append("medo")
        if labels_num[i] == 21.0:
            labels.append("nojo")
        if labels_num[i] == 22.0:
            labels.append("raiva")
        i += 1
    return labels

In [12]:
matrix = kfold(features, labels)

In [13]:
print ("      Classifiers: \t Accuracy (standard deviation)".expandtabs(37))
j = 0
for j in range(9):
    cl = matrix.iloc[0:,j] # separar todos os classificadores para depois fazer suas medias e desvios.
    #print(cl)
    #print("media", cl.mean())
    #print("desvio padrão", cl.std())
    print ("{:41} {:.3f} (+/- {:.3f}) ".format(names[j], cl.mean(), cl.std() * 2))

      Classifiers:                    Accuracy (standard deviation)
SVC with linear kernel                    0.784 (+/- 0.070) 
SVC with RBF kernel                       0.190 (+/- 0.056) 
SVC with polynomial (degree 3) kernel     0.797 (+/- 0.065) 
K Nearest Neighbors                       0.532 (+/- 0.036) 
Decision Tree                             0.490 (+/- 0.064) 
Random Forest                             0.491 (+/- 0.067) 
AdaBoost                                  0.460 (+/- 0.041) 
Naive Bayes                               0.421 (+/- 0.040) 
OPF                                       0.544 (+/- 0.041) 


# A partir daqui vou fazer o One hot encoding

In [14]:
y_oh = pd.get_dummies(labels) # aqui eu binarizo as labels e aplico o One Hot Encoding.
print(y_oh)

      0.0   2.0   9.0   12.0  13.0  21.0  22.0
0        0     0     0     0     0     1     0
1        0     0     0     0     0     1     0
2        0     0     0     1     0     0     0
3        0     0     0     0     1     0     0
4        0     0     0     0     1     0     0
5        0     0     0     1     0     0     0
6        0     1     0     0     0     0     0
7        1     0     0     0     0     0     0
8        1     0     0     0     0     0     0
9        0     0     0     1     0     0     0
10       0     0     1     0     0     0     0
11       1     0     0     0     0     0     0
12       0     0     0     0     1     0     0
13       0     0     0     1     0     0     0
14       0     0     0     0     0     0     1
15       0     0     1     0     0     0     0
16       0     0     0     0     0     1     0
17       0     0     0     0     0     0     1
18       1     0     0     0     0     0     0
19       0     0     1     0     0     0     0
20       0   

As duas próximas células foram feitas em teste para ver o que acontece quando tento classificar com 7 colunas de labels. Deu pra ver que os classificadores do scikit learn querem apenas uma coluna de label.  

In [17]:
label = y_oh
label = label.to_numpy()
label = label.astype('float64')
print(label)

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [18]:
matrix = kfold(features, label)

ValueError: bad input shape (1683, 7)

Porém, se eu mando só uma coluna da matriz "binarizada" (y_oh), o resultado sai muito bom, até difícil de acreditar. Deve-se atentar para a ideia de mandar uma coluna binária de cada vez e depois fazer algum tipo de média ou deixar da forma que está (números puros das emoções - 0, 2, 9, 12, 13, 21, 22). 

In [19]:
y_oh = pd.get_dummies(labels) # aqui eu binarizo as labels e aplico o One Hot Encoding.
print(y_oh)

      0.0   2.0   9.0   12.0  13.0  21.0  22.0
0        0     0     0     0     0     1     0
1        0     0     0     0     0     1     0
2        0     0     0     1     0     0     0
3        0     0     0     0     1     0     0
4        0     0     0     0     1     0     0
5        0     0     0     1     0     0     0
6        0     1     0     0     0     0     0
7        1     0     0     0     0     0     0
8        1     0     0     0     0     0     0
9        0     0     0     1     0     0     0
10       0     0     1     0     0     0     0
11       1     0     0     0     0     0     0
12       0     0     0     0     1     0     0
13       0     0     0     1     0     0     0
14       0     0     0     0     0     0     1
15       0     0     1     0     0     0     0
16       0     0     0     0     0     1     0
17       0     0     0     0     0     0     1
18       1     0     0     0     0     0     0
19       0     0     1     0     0     0     0
20       0   

In [ ]:
i = 0
emotion = []
for i in range(8):
    label = y_oh.iloc[0:,i]
    label = label.to_numpy()
    matrix = kfold(features, label)
    results(matrix)

      Classifiers:                    Accuracy (standard deviation)
SVC with linear kernel                    0.949 (+/- 0.018) 
SVC with RBF kernel                       0.876 (+/- 0.033) 
SVC with polynomial (degree 3) kernel     0.954 (+/- 0.024) 
K Nearest Neighbors                       0.890 (+/- 0.036) 
Decision Tree                             0.867 (+/- 0.046) 
Random Forest                             0.877 (+/- 0.030) 
AdaBoost                                  0.901 (+/- 0.036) 
Naive Bayes                               0.682 (+/- 0.073) 
OPF                                       0.874 (+/- 0.042) 
      Classifiers:                    Accuracy (standard deviation)
SVC with linear kernel                    0.955 (+/- 0.023) 
SVC with RBF kernel                       0.872 (+/- 0.046) 
SVC with polynomial (degree 3) kernel     0.948 (+/- 0.022) 
K Nearest Neighbors                       0.916 (+/- 0.040) 
Decision Tree                             0.906 (+/- 0.021) 
Random For

In [20]:
def results(matrix1):
    print ("      Classifiers: \t Accuracy (standard deviation)".expandtabs(37))
    j = 0
    for j in range(9):
        cl = matrix1.iloc[0:,j] # separar todos os classificadores para depois fazer suas medias e desvios.
        #print(cl)
        #print("media", cl.mean())
        #print("desvio padrão", cl.std())
        print ("{:41} {:.3f} (+/- {:.3f}) ".format(names[j], cl.mean(), cl.std() * 2))